# CSV to GeoBlacklight JSON

### This script takes an input CSV of metadata and converts it to a GeoBlacklight JSON

#### Look in the repo for an example input CSV, and let's get started
Import necessary modules

In [1]:
import csv
import json
import os
from datetime import datetime

This is a dictionary to translate single-value Dublin Core/GBL fields into GBLJson

In [2]:
single_dict = {
    "Identifier":["layer_slug_s","dc_identifier_s"],
    "Status":["b1g_status_s"],
    "Code":["b1g_code_s"],
    "Date Accessioned":["b1g_dateAccessioned_s"],
    "Date Retired":["b1g_dateRetired_s"],
    "Suppressed":["suppressed_b"],
    "Accrual Method":["dct_accrualMethod_s"],
    "Title":["dc_title_s"],
    "Description":["dc_description_s"],
    "Date Issued":["dct_issued_s"],
    "Solr Year":["solr_year_i"],
    "Provenance":["dct_provenance_s"],
    "Format":["dc_format_s"],
    "Geometry Type":["layer_geom_type_s"],
    "Image":["b1g_image_ss"],
    "Layer ID":["layer_id_s"]
    }

And this is a dictionary to translate multivalue Dublin Core/GBL fields into GBLJson

In [3]:
multiple_dict = {
    "Is Part Of":["dct_isPartOf_sm"],
    "Alternative Title":["dct_alternativeTitle_sm"],
    "Genre":["b1g_genre_sm"],
    "Subject":["dc_subject_sm"],
    "Keyword":["b1g_keyword_sm"],
    "Temporal Coverage":["dct_temporal_sm"],
    "Spatial Coverage":["dct_spatial_sm"],
    "Publisher":["dc_publisher_sm"],
    "Creator":["dc_creator_sm"],
    "Language":["dc_language_sm"],
    "Type":["dc_type_sm"],
    "Geonames":["b1g_geonames_sm"]
    }

This statement will create a folder to store the jsons if one does not already exist

In [4]:
if not os.path.exists("json"):
    os.mkdir("json")

Open the CSV with the GBL data. Change the string inside the open statement to match your file name

In [5]:
csvfile = open('template_from_master.csv', 'r')

Reads the CSV into a dictionary and sets the date modified to today

In [ ]:
reader = csv.DictReader(csvfile)
date_modified = datetime.today().strftime('%Y-%m-%d')

Now this is where the work happens. 
* Each row within the reader is a dictionary containing one line of the CSV. <br>
* A starting dictionary is created that has some pre-populated default values. These can change as needed; feel free to modify them. <br>
* Each row is examined for an identifying code. This code separates the records into collections. A folder for each code is created in the json folder so that the jsons can be sorted into their respective collection. <br>
* The script then goes through the single and multiple dictionaries that were defined above and writes them into the starting dictionary. <br>
* Next, the script looks for the the spatial coverage field and splits the WSEN values into their own variables. A centroid is calculated, and the geometry and centroid fields are populated accordingly. If the spatial coverage field doesn't have all of the necessary values, then the geometry and centroid fields are written to be null. <br>
* The references field needs to be created, so the script looks for relevant fields, writes them to a list if they have a value, and then formats and adds the list to the small dictionary before writing the json file.
* Finally, the unique identifier is pulled out, the output filename is named according to that unique identifier, and the output json file is written. This happens for every row in the CSV, so each record will be written to its own JSON file.

In [7]:
for row in reader: #each row is a dictionary
    code = ""
    ref = []
    small_dict = {"geoblacklight_version":"1.0","dc_rights_s":"Public","layer_modified_dt":date_modified} #starting dictionary with set values
    for key,val in row.items():
        if key == "Code":
            code = val
            if not os.path.exists("json/" + val): #makes a new folder for each code
                os.mkdir("json/" + val)
        if key in single_dict:
            for fieldname in single_dict[key]:
                small_dict[fieldname] = val
        if key in multiple_dict:
            for fieldname in multiple_dict[key]:
                small_dict[fieldname] = val.split('|') #creates a list with the multiple values
        if key == "Bounding Box":
            val = val.split(',')
            if len(val) == 4: #takes care of bounding box values and calculates centroid
                west = val[0]
                south = val[1]
                east = val[2]
                north = val[3]
                centerlat = (float(north)+float(south))/2
                centerlong = (float(east)+float(west))/2
                small_dict["solr_geom"] = "ENVELOPE("+west+","+east+","+north+","+south+")"
                small_dict["b1g_centroid_ss"] = str(centerlat) + "," + str(centerlong)
            else: #if the bounding box doesn't have all coordinates, just write values as null
                small_dict["solr_geom"] = "NULL"
                small_dict["b1g_centroid_ss"] = "NULL"
        if key == "Information" and val != '':
            to_append = '"http://schema.org/url":"' + val + '"'
            #print(to_append)
            ref.append(to_append)
        if key == "Download" and val != '':
            to_append = '"http://schema.org/downloadUrl":"' + val + '"'
            ref.append(to_append)
        if key == "MapServer" and val != '':
            to_append = '"urn:x-esri:serviceType:ArcGIS#DynamicMapLayer":"' + val + '"'
            ref.append(to_append)
        if key == "FeatureServer" and val != '':
            to_append = '"urn:x-esri:serviceType:ArcGIS#FeatureLayer":"' + val + '"'
            ref.append(to_append)
        if key == "ImageServer" and val != '':
            to_append = '"urn:x-esri:serviceType:ArcGIS#ImageMapLayer":"' + val + '"'
            ref.append(to_append)
        if key == "ISO Metadata" and val != '':
            to_append = '"http://www.isotc211.org/schemas/2005/gmd/":"' + val + '"'
            ref.append(to_append)
        if key == "FGDC Metadata" and val != '':
            to_append = '"http://www.opengis.net/cat/csw/csdgm":"' + val + '"'
            ref.append(to_append)
        if key == "WFS" and val != '':
            to_append = '"http://www.opengis.net/def/serviceType/ogc/wfs":"' + val + '"'
            ref.append(to_append)
        if key == "WMS" and val != '':
            to_append = '"http://www.opengis.net/def/serviceType/ogc/wms":"' + val + '"'
            ref.append(to_append)
        if key == "HTML" and val != '':
            to_append = '"http://www.w3.org/1999/xhtml":"' + val + '"'
            ref.append(to_append)
        if key == "IIIF" and val != '':
            to_append = '"http://iiif.io/api/image":"' + val + '"'
            ref.append(to_append)
        if key == "Manifest" and val != '':
            to_append = '"http://iiif.io/api/presentation#manifest":"' + val + '"'
            ref.append(to_append)
        if key == "IndexMaps" and val != '':
            to_append = '"https://openindexmaps.org":"' + val + '"'
            ref.append(to_append)
    small_dict["dct_references_s"] = '{' + (','.join(ref)) + '}'
    iden = row['Identifier']
    filename = iden + ".json"
    with open("json/"+code+"/"+filename, 'w') as jsonfile: #writes to a json with the identifier as the filename
        json.dump(small_dict,jsonfile,indent=2)